In [1]:
# Import h2o library
import h2o
from h2o.estimators import H2OKMeansEstimator
# Importing dataset
import pandas as pd
import numpy as np
import os
import io
import glob
import pickle
import seaborn as sns #visualisation
import matplotlib.pyplot as plt #visualisation
import requests
import re
import multiprocessing as mp
%matplotlib inline 
sns.set(color_codes=True)
os.chdir('/home/jupyter-chandler.vaughn/data/')



Bad key "text.kerning_factor" on line 4 in
/opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
h2o.init(max_mem_size="96G", nthreads = 31)
#nthreads = 30

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /opt/tljh/user/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpbr9jo077
  JVM stdout: /tmp/tmpbr9jo077/h2o_jupyter_chandler_vaughn_started_from_python.out
  JVM stderr: /tmp/tmpbr9jo077/h2o_jupyter_chandler_vaughn_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,1 month and 29 days
H2O_cluster_name:,H2O_from_python_jupyter_chandler_vaughn_jn987p
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,96 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,31
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
files = ['unmuted_epoch_alert_90days_sparse_parallel_final_noWIP_0_3000000.pickle',
         'unmuted_epoch_alert_90days_sparse_parallel_final_noWIP_3000001_6000000.pickle',
         'unmuted_epoch_alert_90days_sparse_parallel_final_noWIP_6000001_9000000.pickle',
         'unmuted_epoch_alert_90days_sparse_parallel_final_noWIP_9000001_10921397.pickle',
         'unmuted_epoch_alert_90days_sparse_parallel_final_noWIP_3mm.pickle']

dataset = pd.DataFrame()

for file in files:
    load = pd.read_pickle(file)
    dataset = pd.concat([dataset,load])

dataset.reset_index(inplace=True)
dataset = dataset.drop('index', axis=1)
dataset['date'] = pd.to_datetime(dataset['date'],unit='s')
dataset.head()

,date,count,host_ip,alertId,alertName,bosunAlertId,scalingFactor,displayText,resolveOnNoData,source_instance_uuid,...,goal_type,is_cvm,power_state,vm_annotation,vm_type,cluster_uuid,endpoint,ip_addresses,client_addr,severity
0,2020-03-16 15:59:59,14,anc.cell-1.iad-1-dev.us-east-1a.ntnxi.net:4801,0e66461d-179f-4070-89b4-cde40013dc4b,Xi Networking: OVN Duplicate Chassis,0e66461d-179f-4070-89b4-cde40013dc4b/ad9d0c5a65,1,,false,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-03-16 15:59:59,14,NaN,ce1ed53f-dca9-4f7e-8a32-b16a712ee5ab,DCinfra DNS Service Health,ce1ed53f-dca9-4f7e-8a32-b16a712ee5ab/5d447ccdfd,1,,false,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-03-16 15:59:59,14,NaN,c313fd3b-8c3b-4691-85bd-74316937d13c,AHV Failed Node SSH Prod: AHV Failed Node SSH ...,c313fd3b-8c3b-4691-85bd-74316937d13c/b5198e3e30,1,,false,4542a58a-65c1-5de9-916e-12132f9758b4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-03-16 15:59:59,14,NaN,c313fd3b-8c3b-4691-85bd-74316937d13c,AHV Failed Node SSH Prod: AHV Failed Node SSH ...,c313fd3b-8c3b-4691-85bd-74316937d13c/b5198e3e30,1,,false,ce69c94d-8252-5414-a7f7-5f4524218db2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-03-16 15:59:59,14,NaN,ce1ed53f-dca9-4f7e-8a32-b16a712ee5ab,DCinfra DNS Service Health,ce1ed53f-dca9-4f7e-8a32-b16a712ee5ab/5d447ccdfd,1,,false,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data_h2o = h2o.H2OFrame(dataset)

Parse progress: |██████████████████████████████████████████████████████████| 100%


In [5]:
del dataset

In [7]:
# run h2o Kmeans to estimate good start points
h2o_km = H2OKMeansEstimator(k=5, init="furthest", standardize=True)


h2o_km.train(training_frame=data_h2o)


user_points = h2o.H2OFrame(h2o_km.centers())

# show details
h2o_km.show()


kmeans Model Build progress: |████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1594676147338_1


Model Summary: 


,,number_of_rows,number_of_clusters,number_of_categorical_columns,number_of_iterations,within_cluster_sum_of_squares,total_sum_of_squares,between_cluster_sum_of_squares
0,,13220625.0,5.0,62.0,10.0,4.290744e+07,6.952473e+07,2.661729e+07




ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 42532876.637107044
Total Sum of Square Error to Grand Mean: 69524734.87565318
Between Cluster Sum of Square Error: 26991858.238546133

Centroid Statistics: 


,,centroid,size,within_cluster_sum_of_squares
0,,1.0,9369895.0,2.211691e+07
1,,2.0,641789.0,3.523314e+06
2,,3.0,1201398.0,6.411508e+06
3,,4.0,592877.0,3.199445e+06
4,,5.0,1414666.0,7.281702e+06



Scoring History: 


,,timestamp,duration,iterations,number_of_reassigned_observations,within_cluster_sum_of_squares
0,,2020-07-13 22:03:09,0.032 sec,0.0,NaN,NaN
1,,2020-07-13 22:03:49,40.320 sec,1.0,13220625.0,5.350935e+07
2,,2020-07-13 22:03:57,48.528 sec,2.0,636666.0,4.780834e+07
3,,2020-07-13 22:04:05,55.719 sec,3.0,450135.0,4.710720e+07
4,,2020-07-13 22:04:11,1 min 1.868 sec,4.0,535251.0,4.660225e+07
5,,2020-07-13 22:04:16,1 min 7.335 sec,5.0,412338.0,4.568991e+07
6,,2020-07-13 22:04:22,1 min 13.042 sec,6.0,125352.0,4.540185e+07
7,,2020-07-13 22:04:27,1 min 18.488 sec,7.0,124228.0,4.535459e+07
8,,2020-07-13 22:04:33,1 min 23.925 sec,8.0,241093.0,4.526298e+07
9,,2020-07-13 22:04:39,1 min 30.208 sec,9.0,575358.0,4.439967e+07


In [10]:
h2o_km.summary

Model Details
H2OKMeansEstimator :  K-means
Model Key:  KMeans_model_python_1594676147338_1


Model Summary: 


,,number_of_rows,number_of_clusters,number_of_categorical_columns,number_of_iterations,within_cluster_sum_of_squares,total_sum_of_squares,between_cluster_sum_of_squares
0,,13220625.0,5.0,62.0,10.0,4.290744e+07,6.952473e+07,2.661729e+07




ModelMetricsClustering: kmeans
** Reported on train data. **

MSE: NaN
RMSE: NaN
Total Within Cluster Sum of Square Error: 42532876.637107044
Total Sum of Square Error to Grand Mean: 69524734.87565318
Between Cluster Sum of Square Error: 26991858.238546133

Centroid Statistics: 


,,centroid,size,within_cluster_sum_of_squares
0,,1.0,9369895.0,2.211691e+07
1,,2.0,641789.0,3.523314e+06
2,,3.0,1201398.0,6.411508e+06
3,,4.0,592877.0,3.199445e+06
4,,5.0,1414666.0,7.281702e+06



Scoring History: 


,,timestamp,duration,iterations,number_of_reassigned_observations,within_cluster_sum_of_squares
0,,2020-07-13 22:03:09,0.032 sec,0.0,NaN,NaN
1,,2020-07-13 22:03:49,40.320 sec,1.0,13220625.0,5.350935e+07
2,,2020-07-13 22:03:57,48.528 sec,2.0,636666.0,4.780834e+07
3,,2020-07-13 22:04:05,55.719 sec,3.0,450135.0,4.710720e+07
4,,2020-07-13 22:04:11,1 min 1.868 sec,4.0,535251.0,4.660225e+07
5,,2020-07-13 22:04:16,1 min 7.335 sec,5.0,412338.0,4.568991e+07
6,,2020-07-13 22:04:22,1 min 13.042 sec,6.0,125352.0,4.540185e+07
7,,2020-07-13 22:04:27,1 min 18.488 sec,7.0,124228.0,4.535459e+07
8,,2020-07-13 22:04:33,1 min 23.925 sec,8.0,241093.0,4.526298e+07
9,,2020-07-13 22:04:39,1 min 30.208 sec,9.0,575358.0,4.439967e+07


<bound method ModelBase.summary of >